## Adjacency list construction -- example : EU NUTS2 regions
shp download : https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/nuts <br>
original idea developed by Eszter Bokányi (bokae on github)

In [38]:
import geopandas as gpd
import matplotlib.pyplot as plt

In [39]:
# NUTS-3 level shapefile
map_df = gpd.read_file('../data/shapefile/NUTS_RG_10M_2016_3035.shp')

# FILTER for NUTS-2
map_df = map_df[map_df['LEVL_CODE']<3]

# keep 4-digit region codes
map_df['digits'] = map_df['NUTS_ID'].apply(lambda x: len(x))
map_df = map_df[map_df['digits'] == 4]

In [40]:
# create a spatial index for faster join
map_df["buffered_geometry"] = map_df.buffer(250)
si = map_df.set_geometry("buffered_geometry").sindex

In [41]:
# possible matches based on spatial index
map_df["possible_matches"] = map_df.buffer(250).map(lambda p: list(si.intersection(p.bounds)))

# exact matches calculated only for possible matches, excluding the polygon in question
map_df["neighbors"] = map_df.apply(lambda row: [map_df.index[e] for e in row["possible_matches"] if row["buffered_geometry"].intersects(map_df["buffered_geometry"].iloc[e]) and map_df.index[e]!=row.name], axis=1)

In [42]:
# construct the final dataframe
adj_df = map_df.loc[:,['NUTS_ID', 'neighbors']].dropna()
adj_df = adj_df[adj_df['neighbors'].map(len) >0]
adj_df = adj_df.explode('neighbors')
adj_df['neighbors'] = adj_df['neighbors'].map(map_df['NUTS_ID'].to_dict())

In [24]:
# save the final adjacency list
adj_df.to_csv('../data/adj_list_NUTS2.csv', sep=';')

In [ ]:
# NUTS-3 level shapefile
map_df = gpd.read_file('../data/shapefile/NUTS_RG_10M_2016_3035.shp/NUTS_RG_10M_2016_3035.shp')

# FILTER for NUTS-2
map_df = map_df[map_df['STAT_LEVL_']<3]
# map_df['geometry'] = map_df['geometry'].simplify(tolerance=0.2)

# keep 4-digit region codes
map_df['digits'] = map_df['NUTS_ID'].apply(lambda x: len(x))
map_df = map_df[map_df['digits'] ==4]

In [ ]:
# lat,lon coordinate system code
init_crs = 4326
# cartesian coordinate system code
project_crs = 3857

# projecting census tract geometries into the same Cartesian coordinate system as the user coordinates
map_df.crs = {'init': 'epsg:' + str(init_crs)}
map_df.to_crs(epsg = project_crs, inplace=True)

In [ ]:
map_df.boundary.plot(figsize=(20,20))
#plt.xlim([-12,35])
#plt.ylim([30,75])

In [ ]:
# creating a spatial index for faster join
map_df["buffered_geometry"] = map_df.buffer(250)
si = map_df.set_geometry("buffered_geometry").sindex

In [ ]:
# possible matches based on spatial index
map_df["possible_matches"] = map_df.buffer(250).map(lambda p: list(si.intersection(p.bounds)))
# exact matches calculated only for possible matches, excluding the polygon in question
map_df["neighbors"] = map_df.apply(lambda row: [map_df.index[e] for e in row["possible_matches"] if row["buffered_geometry"].intersects(map_df["buffered_geometry"].iloc[e]) and map_df.index[e]!=row.name],axis=1)

In [ ]:
# for plotting
def plot_neighbors(index):
    fig = plt.figure(figsize=(20,20))
    ax = plt.gca()
    map_df.boundary.plot(ax=ax)
    map_df.loc[[index],"geometry"].plot(ax=ax,color="black")
    map_df.loc[map_df.loc[index]["neighbors"]].plot(ax=ax,color="red")
    plt.ylim(0.4e7, 1e7)
    plt.xlim(-2e6, 2e6)
    plt.show()

In [ ]:
from ipywidgets import interact
interact(plot_neighbors,index=map_df.index.tolist())

In [ ]:
# get rows by index
neighbors_df = map_df.loc[:,['NUTS_ID', 'neighbors']].dropna()
print(neighbors_df.shape)
neighbors_df = neighbors_df[neighbors_df['neighbors'].map(len) >0]
print(neighbors_df.shape)
neighbors_df = neighbors_df.explode('neighbors')

In [ ]:
neighbors_df['neighbors'] = neighbors_df['neighbors'].map(map_df['NUTS_ID'].to_dict())

In [ ]:
neighbors_df.to_csv('../data/neighbors_NUTS2.csv', sep=';')